### Metadata source

In [6]:
import sys
import os
import pandas as pd
import json

from prompt_toolkit.input.vt100 import raw_mode
from tqdm import tqdm
from dotenv import load_dotenv
from pydantic import BaseModel
from concurrent.futures import ThreadPoolExecutor, as_completed
from langchain_core.rate_limiters import InMemoryRateLimiter

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.articles import create_static_metadata, create_keywords_tags_fuzzy, keywords_dict
# from src.llm import get_gemini_llm_client
# from src.prompts import get_metadata_prompt

if not load_dotenv():
    raise Exception('Error loading .env file. Make sure to place a valid OPEN_AI_KEY in the .env file.')

Setup the paths to data sources

In [7]:
ARTICLES_CLEAN_DIR = os.path.join("..", "data", "articles_clean")
METADATA_PATH = os.path.join("..", "data", "metadata.csv")

Extract metadata

In [9]:
articles = os.listdir(ARTICLES_CLEAN_DIR)
import random
random.seed(42)
random.shuffle(articles)
# with open(, "r", encoding="utf-8") as file:
#     article = json.load(file)

article_ls = []
for name in articles[:100]:
    with open(os.path.join(ARTICLES_CLEAN_DIR, name), "r", encoding="utf-8") as file:
        article = json.load(file)

    article_ls.append(article)

article_df = pd.DataFrame(article_ls)

In [10]:
article_df

,id,published_at,author,title,category,ressort,text
0,948de0b1-b3b7-4c45-b22a-a074d3761cfc,2010-06-09 18:41,Walter Hämmerle,Unerbittliche Unvernunft,Leitartikel,Meinung,In Deutschland gibt es einige. Einer von ihnen...
1,d3da5c1b-9f10-4603-a648-d511cee9c14e,2016-02-17 17:50,WZ-Korrespondentin Martyna Czarnowska,Englisches Frühstück,Politik,Nachrichten,"Brüssel. ""Ein ""englisches Frühstück"": Das könn..."
2,a23cee8a-a2e3-43b7-b62b-6e5c3c0f2e87,2008-03-26 18:57,Helmut Dité,Gewinne sprudeln im Osten,Wirtschaft,Nachrichten,Im Gegenteil: Nachdem der Gewinn im CEE-Raum 2...
3,9fa0d59f-06ed-434f-9a7e-aabeff65fcbf,2015-12-17 18:15,Michael Schmölzer,Starke Ansage - leere Drohung?,Politik,Nachrichten,Wien/Berlin/Brüssel. Im Streit um die EU-weite...
4,dda1a4cf-76b9-41b0-a637-d0b7e12625d9,2022-11-14 09:30,Peter De Coensel,Wirtschaftspolitik als Waffe,Gastkommentare,Meinung,Von einer vorübergehenden Inflation im Zusamme...
5,0cf87a14-3d90-43f3-9659-c5e3f230add2,2019-09-18 11:00,Jan Michael Marchart,Das Jobmigranten-Karussell,Politik,Nachrichten,Wer es bis dahin nicht für möglich gehalten ha...
6,68990817-3939-4518-845c-5d7e5c0057a2,2017-06-07 17:49,Eva Stanzl,Die Bewahrung des Mythos,Wissen,Nachrichten,Ein Dutzend Journalisten inspiziert das hinter...
7,0cf4b22c-931e-4438-b7c1-620634b9bb01,2011-09-12 18:21,Stefan Melichar,Justiz ermittelt gegen Ex-Hypo-Berater,Wirtschaft,Nachrichten,Wien.\nDie - ohnehin schon ziemlich umfangreic...
8,de9c2c3e-4d94-4c79-92af-8d85a1f9a7cf,2007-04-17 17:48,WZ-Korrespondentin Alexandra Klausmann,Streik bei Skoda legt Produktion lahm,Wirtschaft,Nachrichten,"Ziel des Streiks, so Jaroslav Povsik, Vorsitze..."
9,d1c08103-21e1-4104-9c84-bcb776b0609c,2015-02-25 18:06,Brigitte Pechar,Grünes Misstrauen,Politik,Nachrichten,"Wien. Hypo-Untersuchungsausschuss, Islamgesetz..."


In [42]:
from nltk.corpus import stopwords
import string

stop_words = set(stopwords.words("english"))

# TODO
def clean_text(text, max_length=10000):
    text = text.lower()
    text = "".join([char if char not in string.punctuation else " " for char in text])
    words = text.split()
    words = [word for word in words if word not in stop_words]
    text = " ".join(words)
    return text[:max_length]

In [24]:
article_df["text_cleaned"] = article_df["text"].apply(clean_text)

In [41]:
article_df

,id,published_at,author,title,category,ressort,text,text_cleaned
0,948de0b1-b3b7-4c45-b22a-a074d3761cfc,2010-06-09 18:41,Walter Hämmerle,Unerbittliche Unvernunft,Leitartikel,Meinung,In Deutschland gibt es einige. Einer von ihnen...,deutschland gibt es einige einer von ihnen ist...
1,d3da5c1b-9f10-4603-a648-d511cee9c14e,2016-02-17 17:50,WZ-Korrespondentin Martyna Czarnowska,Englisches Frühstück,Politik,Nachrichten,"Brüssel. ""Ein ""englisches Frühstück"": Das könn...",brüssel ein englisches frühstück das könnte na...
2,a23cee8a-a2e3-43b7-b62b-6e5c3c0f2e87,2008-03-26 18:57,Helmut Dité,Gewinne sprudeln im Osten,Wirtschaft,Nachrichten,Im Gegenteil: Nachdem der Gewinn im CEE-Raum 2...,im gegenteil nachdem der gewinn im cee raum 20...
3,9fa0d59f-06ed-434f-9a7e-aabeff65fcbf,2015-12-17 18:15,Michael Schmölzer,Starke Ansage - leere Drohung?,Politik,Nachrichten,Wien/Berlin/Brüssel. Im Streit um die EU-weite...,wien berlin brüssel im streit um die eu weite ...
4,dda1a4cf-76b9-41b0-a637-d0b7e12625d9,2022-11-14 09:30,Peter De Coensel,Wirtschaftspolitik als Waffe,Gastkommentare,Meinung,Von einer vorübergehenden Inflation im Zusamme...,von einer vorübergehenden inflation im zusamme...
5,0cf87a14-3d90-43f3-9659-c5e3f230add2,2019-09-18 11:00,Jan Michael Marchart,Das Jobmigranten-Karussell,Politik,Nachrichten,Wer es bis dahin nicht für möglich gehalten ha...,wer es bis dahin nicht für möglich gehalten ha...
6,68990817-3939-4518-845c-5d7e5c0057a2,2017-06-07 17:49,Eva Stanzl,Die Bewahrung des Mythos,Wissen,Nachrichten,Ein Dutzend Journalisten inspiziert das hinter...,ein dutzend journalisten inspiziert das hinter...
7,0cf4b22c-931e-4438-b7c1-620634b9bb01,2011-09-12 18:21,Stefan Melichar,Justiz ermittelt gegen Ex-Hypo-Berater,Wirtschaft,Nachrichten,Wien.\nDie - ohnehin schon ziemlich umfangreic...,wien die ohnehin schon ziemlich umfangreiche b...
8,de9c2c3e-4d94-4c79-92af-8d85a1f9a7cf,2007-04-17 17:48,WZ-Korrespondentin Alexandra Klausmann,Streik bei Skoda legt Produktion lahm,Wirtschaft,Nachrichten,"Ziel des Streiks, so Jaroslav Povsik, Vorsitze...",ziel des streiks jaroslav povsik vorsitzender ...
9,d1c08103-21e1-4104-9c84-bcb776b0609c,2015-02-25 18:06,Brigitte Pechar,Grünes Misstrauen,Politik,Nachrichten,"Wien. Hypo-Untersuchungsausschuss, Islamgesetz...",wien hypo untersuchungsausschuss islamgesetz u...


In [8]:
# Articles length statistics
df_metadata["words_count"].describe()

count    87754.000000
mean       511.983670
std        351.851569
min          1.000000
25%        281.000000
50%        422.000000
75%        632.000000
max       5931.000000
Name: words_count, dtype: float64

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from tqdm.auto import tqdm
import pandas as pd

# Candidate labels to evaluate
candidate_labels = ['financial crisis', 'sustainability', 'fake news', 'AI',
                    'digitalization', 'local journalism', 'covid', 'demographics', 'innovation']

# Device and model/tokenizer setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
nli_model = AutoModelForSequenceClassification.from_pretrained('joeddav/xlm-roberta-large-xnli').to(device)
tokenizer = AutoTokenizer.from_pretrained('joeddav/xlm-roberta-large-xnli')

# Define batch size for efficient GPU usage
batch_size = 32  # adjust as needed

# Loop over each candidate label, computing probabilities in batches
for label in candidate_labels:
    hypothesis = f'This example is {label}.'
    prob_list = []

    # Iterate over the DataFrame in batches
    for start in tqdm(range(0, len(article_df), batch_size), desc=f"Processing label: {label}"):
        end = start + batch_size
        batch_texts = article_df['text_cleaned'].iloc[start:end].tolist()

        # Tokenize the premises with the same hypothesis for the batch
        inputs = tokenizer(batch_texts, [hypothesis] * len(batch_texts),
                           return_tensors='pt', truncation=True, padding=True)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad():
            logits = nli_model(**inputs).logits

        # Select logits for "contradiction" (index 0) and "entailment" (index 2)
        entail_contradiction_logits = logits[:, [0, 2]]
        probs = torch.softmax(entail_contradiction_logits, dim=1)[:, 1]
        prob_list.append(probs.cpu())

    # Concatenate the batch results and add as a new column to the dataframe
    all_probs = torch.cat(prob_list).numpy()
    article_df[label] = all_probs

# Now article_df has one additional column per candidate label containing the probability that the label is true.

Categories by WZ

In [9]:
df_metadata["category"].describe()

count       87754
unique         47
top       Politik
freq        31919
Name: category, dtype: object

In [10]:
df_metadata["category"].value_counts()

category
Politik                                  31919
Wirtschaft                               16512
Kommentare                               11408
Gastkommentare                            7253
Wissen                                    5632
Europaarchiv                              5341
Leitartikel                               3292
Analysen                                  2478
Reflexionen                               2308
Recht                                      652
Auf Justitias Spuren                       196
Leserforum                                 170
Klimawandel                                 52
Wiener Zeitung - seit 1703                  47
Sterbehilfe                                 44
1914                                        39
100 Jahre Republik                          38
Stadtentwicklung                            28
Wald                                        27
100 Jahre Verfassung                        26
EU für mich                                 26
Asyl

In [11]:
# Tags statistics (only available if with_tags=True)
df_metadata.explode("tags")["tags"].value_counts()

tags
other               73976
COVID                5415
Fake News            4610
Digitalization       1149
Demographics          990
Innovation            823
Financial Crises      612
Sustainability        138
AI                     37
Local Journalism        4
Name: count, dtype: int64

In [12]:
# Missing authors
df_metadata["author"].isnull().sum()

np.int64(77)

Metadata df

In [13]:
df_metadata.head(5)

,id,title,author,published_at,words_count,filename,category,section,tags
0,cbbd50ec-c07b-4fcf-b7bf-dd0b7bacc887,100.000 Plätze mehr in zehn Jahren,Alexandra Grass,2003-10-08 00:00,264,100000-platze-mehr-in-zehn-jahren.json,Politik,Nachrichten,other
1,a0799ad0-f4ed-4989-8c9a-afa6e88677c6,1.700 Lehrlinge ohne Ausbildung,Werner Grotte,2004-10-06 00:00,255,1700-lehrlinge-ohne-ausbildung.json,Wirtschaft,Nachrichten,other
2,9866b0a2-9c97-4a49-b5c3-d6803a3f9eac,14 Gemeinden erheben schwere Vorwürfe gegen Ra...,Kid Möchel,2011-06-17 18:28,530,14-gemeinden-erheben-schwere-vorwurfe-gegen-ra...,Wirtschaft,Nachrichten,other
3,1e9ee2dd-f1cc-42c3-a61b-47cd64dd3fca,10.000 syrische Babys - geboren in einem ander...,Maysoon Mohammad Khalaf Al-Hijazat,2017-08-16 13:23,522,10000-syrische-babys-geboren-in-einem-anderen-...,Gastkommentare,Meinung,other
4,4fe39ec3-426f-4608-9def-225229a4a476,1700 Euro steuerfrei: Was der SPÖ-Plan bringt,Karl Ettinger,2019-08-26 18:14,824,1700-euro-steuerfrei-was-der-spo-plan-bringt.json,Politik,Nachrichten,other
